# Análise do vestibular ENEM 2019

## Introduction to Data Science

## The CRISP-DM approach to Data Science Tasks

_________________________

## I. Business Understanding
                    
#### The Business Understanding phase focuses on understanding the objectives and requirements of the project.  
+ Determine business objectives:   
+ Assess situation:  
+ Determine data mining goals:  
+ Produce project plan:  




#### Determine business objectives:   

* Compreender quais aspectos melhor determinam a nota de um aluno no Enem de acordo com a Teoria de Resposta ao Item.

- Observar se existe uma correlação entre as notas de cada uma das 4 áreas do conhecimento e a redação. 

* Entender como é distribuida a educação no país, e quais fatores interferem no desempenho de um estudante.

- Analisar se com as cotas há um acesso mais democrático às instituições públicas de ensino.

#### Assess situation:  

* Sabemos que o ENEM utiliza o TRI (Teoria de Resposta ao Item) que classifica as questões conforme a dificuldade, isto é, cada questão possui um peso pré-determinado que não conhecido do público. Esse sistema é utilizado para evitar "chutes", se um aluno erra uma questão considerada fácil e acerta uma questão considerada difícil, o aluno sofre uma penalidade. Desse modo, mesmo com o gabarito dos alunos não nos é possível determinar a nota correta dos alunos, pretendemos obter o melhor modelo de regrassão para atribuir a nota aos alunos.

- A priori, acreditamos que no Brasil, as escolas privadas e escolas públicas federais obtém um melhor desempenho no ENEM, do que os alunos de escolas públicas estaduais e municipais. Além disso, acreditamos que a locaclização geográfica da escola e da moradia do estudante e a renda familiar influencia no seu desempenho, alunos que moram em situações vuneráveis e|ou possuem baixa renda tendem a obter um desempenho inferior a média nacional. 

#### Determine data mining goals:  

- Obter os dados oficiais do INEP de todos os alunos que fizeram o vestibular no ano de 2019: https://www.gov.br/inep/pt-br/acesso-a-informacao/dados-abertos/microdados/enem onde consta os microdados do ENEM ("Os microdados do Enem são o menor nível de desagregação de dados recolhidos por meio do exame. Eles atendem a demanda por informações específicas ao disponibilizar as provas, os gabaritos, as informações sobre os itens, as notas e o questionário respondido pelos inscritos no Enem" - INEP).

* Obter as notas de corte de algumas universidades públicas brasileiras para analizar a democratização do ensino público.

#### Produce project plan: 

- Limpar os dados, por exemplo, remover os alunos treineiros e alunos que não concluiram o vestibular da base de dados.

* Analisar os nossos dados, obter as médias, os mínimos e os máximos.

- Criar classificadores, se necessário. 

* Determinar as informações que serão utilizadas em cada modelo.

- Observar se existe alguma correlação dentre as informações disponíveis. 

* Encontrar o melhor modelo de regressão para as notas.

- Analisar a questão das cotas e da nota de corte das universidades públicas.

* Reunir as informações obtidas e concluir o trabalho.


## II. Data Understanding  

+ Collect initial data:  
+ Describe data:  
+ Explore data:  
+ Verify data quality:  

## III. Data Preparation

+ Select data:   
+ Clean data:   
+ Construct data/transform data/feature engineering:   
+ Integrate data:   
+ (Re) Format data:   

## IV. Modeling

+ Select modeling techniques: 
+ Generate test design: 
+ Build model: 
+ Assess model:  

## V. Evaluation                    

+ Evaluate results: 
+ Review process: 
+ Determine next steps: 

## VI. Deployment

+ Plan deployment: 
+ Plan monitoring and maintenance: 
+ Produce final report: 
+ Review project: 